In [1]:
import pandas as pd
from matplotlib import pyplot as plt
df = pd.read_csv("churn-7043.csv")

In [2]:
df=df.drop(['customerID'], axis=1) 
num_df = df.select_dtypes(include=['float64', 'int64']).copy()
cat_df = df.select_dtypes(include=['object']).copy()

# Categorical boolean mask
categorical_feature_mask = cat_df.dtypes==object
# filter categorical columns using mask and turn it into a list
categorical_cols = cat_df.columns[categorical_feature_mask].tolist()

import numpy as np
#conData=np.log(0.00001 + 1)
conData=0
cat_df=cat_df.fillna(conData)
num_df=num_df.fillna(conData)
cat_df=cat_df.astype(str)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cat_df[categorical_cols] = cat_df[categorical_cols].apply(lambda col: le.fit_transform(col))

#cat_df[categorical_cols].head(10)


churn=cat_df['churn']
cat_df=cat_df.drop(['churn'], axis=1)


num_df=num_df.fillna(conData)

result_df = pd.concat([num_df, cat_df], axis=1)
np.nan_to_num(result_df)

result_df_op=result_df

X=result_df_op
y=churn

from sklearn.model_selection import train_test_split  
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
X_train=X
X_test=X
y_train=y
y_test=y

In [3]:
#pip install mlencoders
from mlencoders.weight_of_evidence_encoder import WeightOfEvidenceEncoder

enc = WeightOfEvidenceEncoder(cols=X.columns)
X_encoded = enc.fit_transform(X, y)
X_encoded.head()

,SeniorCitizen,tenure,MonthlyCharges,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,TotalCharges
0,-0.156151,1.507375,1.018243,0.019602,-0.389129,0.231141,-0.084284,-0.084284,-0.434427,0.685886,-0.274938,0.576292,0.680487,0.333624,0.340675,0.724536,0.335507,0.829097,0.0
1,-0.156151,-0.467143,0.000000,-0.019409,0.308156,0.231141,0.008838,-0.078011,-0.434427,-0.747185,0.609808,-0.218402,0.680487,0.333624,0.340675,-1.045259,-0.615628,-0.424849,0.0
2,-0.156151,1.085495,1.018243,-0.019409,0.308156,0.231141,0.008838,-0.078011,-0.434427,-0.747185,-0.274938,0.576292,0.680487,0.333624,0.340675,0.724536,0.335507,-0.424849,0.0
3,-0.156151,-1.197331,0.000000,-0.019409,0.308156,0.231141,-0.084284,-0.084284,-0.434427,-0.747185,0.609808,-0.218402,-0.703371,0.333624,0.340675,-1.045259,-0.615628,-0.588090,0.0
4,-0.156151,1.085495,0.325095,0.019602,0.308156,0.231141,0.008838,-0.078011,0.691066,0.685886,0.609808,0.576292,0.680487,0.333624,0.340675,0.724536,0.335507,0.829097,0.0


In [4]:
X_encoded[X_encoded < 0]=0
X_encoded.head()

,SeniorCitizen,tenure,MonthlyCharges,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,TotalCharges
0,0.0,1.507375,1.018243,0.019602,0.000000,0.231141,0.000000,0.0,0.000000,0.685886,0.000000,0.576292,0.680487,0.333624,0.340675,0.724536,0.335507,0.829097,0.0
1,0.0,0.000000,0.000000,0.000000,0.308156,0.231141,0.008838,0.0,0.000000,0.000000,0.609808,0.000000,0.680487,0.333624,0.340675,0.000000,0.000000,0.000000,0.0
2,0.0,1.085495,1.018243,0.000000,0.308156,0.231141,0.008838,0.0,0.000000,0.000000,0.000000,0.576292,0.680487,0.333624,0.340675,0.724536,0.335507,0.000000,0.0
3,0.0,0.000000,0.000000,0.000000,0.308156,0.231141,0.000000,0.0,0.000000,0.000000,0.609808,0.000000,0.000000,0.333624,0.340675,0.000000,0.000000,0.000000,0.0
4,0.0,1.085495,0.325095,0.019602,0.308156,0.231141,0.008838,0.0,0.691066,0.685886,0.609808,0.576292,0.680487,0.333624,0.340675,0.724536,0.335507,0.829097,0.0


In [5]:
X.shape

(7043, 19)

In [6]:
X_train=X_encoded
y_train=y
#Univariate Selection
from sklearn.feature_selection import SelectKBest, chi2
select_feature = SelectKBest(chi2, k=15).fit(X_train, y_train)
selected_features_df = pd.DataFrame({'Feature':list(X_train.columns),
                                     'Scores':select_feature.scores_})
selected_features_df.sort_values(by='Scores', ascending=False)

,Feature,Scores
2,MonthlyCharges,1045.853995
1,tenure,681.896420
15,Contract,376.683060
17,PaymentMethod,353.545901
9,OnlineSecurity,285.453849
12,TechSupport,276.357326
8,InternetService,258.787596
18,TotalCharges,198.307311
10,OnlineBackup,173.231172
11,DeviceProtection,145.036817


In [7]:
y_train=y
x_train_chi = select_feature.transform(X_train)
x_train_chi.shape

(7043, 15)

In [31]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RepeatedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler

cv_method = RepeatedKFold(n_splits=5, 
                          n_repeats=3, 
                          random_state=999)

classifier=GaussianNB();
#params = {
#          "priors" : "None",
#          "var_smoothing" : 1e-9
#}
#create an list of var_smoothing to cross validate
#steps = [1e-8, 1e-7, 1e-6, 1e-5, 1e-4]

params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}

grid = GridSearchCV(estimator=classifier, 
                     param_grid=params_NB, 
                     cv=cv_method,
                     verbose=1, 
                     scoring='accuracy', n_jobs=-1)

grid.fit(x_train_chi, y_train)

 
print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)



Fitting 15 folds for each of 100 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done 152 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-1)]: Done 402 tasks      | elapsed:    9.4s
[Parallel(n_jobs=-1)]: Done 752 tasks      | elapsed:   11.3s
[Parallel(n_jobs=-1)]: Done 1202 tasks      | elapsed:   13.8s


{'var_smoothing': 0.43287612810830584}
GaussianNB(var_smoothing=0.43287612810830584)
0.8158458139772459


[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed:   15.2s finished


In [10]:
{'var_smoothing': 1e-09}
GaussianNB(priors=None, var_smoothing=1e-09)
0.51704


0.51704

In [34]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RepeatedKFold
#Create a Gaussian Classifier
classifier = GaussianNB(priors=None, var_smoothing=0.533669923120631)

classifier.fit(x_train_chi,y_train)


cv_method = RepeatedKFold(n_splits=10, 
                          n_repeats=3, 
                          random_state=999)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 5)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

precision=(tp)/(tp+fp)
print ('precision: ',precision)

recall=(tp)/(tp+fn)
print ('recall: ',recall)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1 - score: ',F1_score)


4674 500 808 1061
pod:  0.5676832530765115
pof:  0.09663703131039815
AUC:  0.7355231108830567
accuracy:  0.8142836859292915
precision:  0.6796925048046124
recall:  0.5676832530765115
F1 - score:  0.6186588921282797


In [12]:
 #accuracy=(tp+tn)/(tp+fn+fp+tn)
accuracy=(26836+27976)/(27976+22462+22726+26836)
print ('accuracy: ',accuracy)

    

accuracy:  0.54812


In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# Specify parameters
c_values = list(np.arange(1, 10))


param_grid = [
    {'C': c_values, 'penalty': ['l1'], 'solver' : ['liblinear'], 'multi_class' : ['ovr']},    
    {'C': c_values, 'penalty': ['l2'], 'solver' : ['liblinear', 'newton-cg', 'lbfgs'], 'multi_class' : ['ovr']}
]
 

grid = GridSearchCV(LogisticRegression(), param_grid, verbose=1, cv=10, n_jobs=-1)
grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

Fitting 10 folds for each of 36 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 152 tasks      | elapsed:    0.9s


{'C': 2, 'multi_class': 'ovr', 'penalty': 'l1', 'solver': 'liblinear'}
LogisticRegression(C=2, multi_class='ovr', penalty='l1', solver='liblinear')
0.8216648533204383


[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:    1.5s finished


In [35]:
#LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
classifier = LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          n_jobs=None, penalty='l2', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)
classifier.fit(x_train_chi, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

precision=(tp)/(tp+fp)
print ('precision: ',precision)

recall=(tp)/(tp+fn)
print ('recall: ',recall)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1 - score: ',F1_score)

4751 423 837 1032
pod:  0.5521669341894061
pof:  0.08175492848859683
AUC:  0.7352060028504046
accuracy:  0.821098963509868
precision:  0.709278350515464
recall:  0.5521669341894061
F1 - score:  0.6209386281588448


In [37]:
#LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
classifier = LogisticRegression()
classifier.fit(X_train, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X_train, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

precision=(tp)/(tp+fp)
print ('precision: ',precision)

recall=(tp)/(tp+fn)
print ('recall: ',recall)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1 - score: ',F1_score)


4753 421 844 1025
pod:  0.5484216158373462
pof:  0.08136838036335524
AUC:  0.7335266177369956
accuracy:  0.8203890387618913
precision:  0.7088520055325035
recall:  0.5484216158373462
F1 - score:  0.6184012066365008


In [16]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier  

# Specify parameters
c_values = list(np.arange(1, 10))
param_grid ={
   'n_neighbors': [3,5,11,19],
   'weights': ['uniform', 'distance'],
   'metric': ['euclidean', 'manhattan']
}

grid = GridSearchCV(KNeighborsClassifier(), param_grid, verbose=1, cv=3, n_jobs=-1)
grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:    1.5s finished


{'metric': 'euclidean', 'n_neighbors': 19, 'weights': 'uniform'}
KNeighborsClassifier(metric='euclidean', n_neighbors=19)
0.8148512351723308


In [17]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm

# Specify parameters

Cs = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
gammas = [0.001, 0.01, 0.1, 1, 1e-3, 1e-4]
param_grid = {'C': Cs, 'gamma' : gammas}

tuned_parameters = [{'kernel': ['rbf', 'linear'], 
                     'gamma': [0.001, 0.01, 0.1, 1, 1e-3, 1e-4], 
                     'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]} ]

grid = GridSearchCV(svm.SVC(), param_grid, verbose=1, cv=10, n_jobs=-1)
grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

Fitting 10 folds for each of 42 candidates, totalling 420 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 152 tasks      | elapsed:   17.4s
[Parallel(n_jobs=-1)]: Done 420 out of 420 | elapsed:  1.3min finished


{'C': 10, 'gamma': 0.1}
SVC(C=10, gamma=0.1)
0.8206701321727918


In [38]:
# KNN
from sklearn.neighbors import KNeighborsClassifier  
classifier = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform' )  
classifier.fit(x_train_chi, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)
accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

precision=(tp)/(tp+fp)
print ('precision: ',precision)

recall=(tp)/(tp+fn)
print ('recall: ',recall)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1 - score: ',F1_score)


4587 587 832 1037
pod:  0.5548421615837347
pof:  0.11345187475840743
AUC:  0.7206951434126636
accuracy:  0.7985233565242085
precision:  0.6385467980295566
recall:  0.5548421615837347
F1 - score:  0.5937589464643573


In [19]:
# KNN
from sklearn.neighbors import KNeighborsClassifier  
classifier = KNeighborsClassifier(n_neighbors=5)  
classifier.fit(X_train, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X_train, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)


4607 567 835 1034
pod:  0.5532370251471375
pof:  0.1095863935059915
AUC:  0.721825315820573
accuracy:  0.8009371006673293


In [20]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}


#clf = RandomForestClassifier()

# Choose some parameter combinations to try
#parameters = {'n_estimators': [4, 6, 9], 
#              'max_features': ['log2', 'sqrt','auto'], 
#              'criterion': ['entropy', 'gini'],
#              'max_depth': [2, 3, 5, 10], 
#              'min_samples_split': [2, 3, 5],
#              'min_samples_leaf': [1,5,8]
#             }

#model_params = {
#    'n_estimators': [50, 150, 250],
#    'max_features': ['sqrt', 0.25, 0.5, 0.75, 1.0],
#    'min_samples_split': [2, 4, 6]
#}

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = RandomForestClassifier(), param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
grid_search.fit(x_train_chi, y_train)

print(grid_search.best_params_)
print(grid_search.best_estimator_)
print(grid_search.best_score_)

Fitting 3 folds for each of 288 candidates, totalling 864 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done 114 tasks      | elapsed:   15.6s
[Parallel(n_jobs=-1)]: Done 317 tasks      | elapsed:   43.5s
[Parallel(n_jobs=-1)]: Done 600 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 864 out of 864 | elapsed:  2.1min finished


{'bootstrap': True, 'max_depth': 100, 'max_features': 3, 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 200}
RandomForestClassifier(max_depth=100, max_features=3, min_samples_leaf=3,
                       min_samples_split=10, n_estimators=200)
0.8228024733206599


In [39]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
classifier =RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=100, max_features=3, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=3, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
classifier.fit(x_train_chi, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


precision=(tp)/(tp+fp)
print ('precision: ',precision)

recall=(tp)/(tp+fn)
print ('recall: ',recall)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1 - score: ',F1_score)

4734 440 835 1034
pod:  0.5532370251471375
pof:  0.08504058755315037
AUC:  0.7340982187969936
accuracy:  0.8189691892659378
precision:  0.7014925373134329
recall:  0.5532370251471375
F1 - score:  0.6186060424768173


In [22]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
#classifier =RandomForestClassifier(n_estimators=100,max_depth=2,random_state=0)
classifier =RandomForestClassifier()
classifier.fit(X_train, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X_train, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

4641 533 849 1020
pod:  0.5457463884430177
pof:  0.10301507537688442
AUC:  0.7213656565330666


In [23]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 


#param_grid={
#    'criterion': ['gini', 'entropy'],
#    'splitter': ['best', 'random'],
#    'min_samples_split' : range(10,500,20),
#    'max_depth': [1, 2, 3, 4, 5, 20],
#    'max_features': [1, 2, 3, 4],
#    'max_leaf_nodes': list(range(2, 100)), 
#    'min_samples_split': [2, 3, 4]
#} 

param_grid={
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random']
} 

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = DecisionTreeClassifier(), param_grid = param_grid, 
                          cv = 10, n_jobs = -1, verbose = 2)
grid_search.fit(x_train_chi, y_train)

print(grid_search.best_params_)
print(grid_search.best_estimator_)
print(grid_search.best_score_)

Fitting 10 folds for each of 4 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  14 out of  40 | elapsed:    0.0s remaining:    0.1s


{'criterion': 'entropy', 'splitter': 'random'}
DecisionTreeClassifier(criterion='entropy', splitter='random')
0.7654426579626048


[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    0.1s finished


In [24]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
classifier.fit(x_train_chi, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\tree\_classes.py:323: FutureWarning: The parameter 'presort' is deprecated and has no effect. It will be removed in v0.24. You can suppress this warning by not passing any value to the 'presort' parameter.
  warnings.warn("The parameter 'presort' is deprecated and has no "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\tree\_classes.py:323: FutureWarning: The parameter 'presort' is deprecated and has no effect. It will be removed in v0.24. You can suppress this warning by not passing any value to the 'presort' parameter.
  warnings.warn("The parameter 'presort' is deprecated and has no "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\tree\_classes.py:323: FutureWarning: The parameter 'presort' is deprecated and has no effect. It will be removed in v0.24. You can suppress this warning by not passing any value to the 'presort' parameter.
  warnings.warn("The parameter 'presort' is deprecated and has no "
C:\ProgramData\Anaconda3\

4389 785 925 944
pod:  0.5050829320492242
pof:  0.15172013915732507
AUC:  0.6766813964459496
accuracy:  0.7572057361919636


In [25]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X_train, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

4343 831 884 985
pod:  0.527019796682718
pof:  0.16061074603788172
AUC:  0.6832045253224182


In [26]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

param_grid = {
    
    }

grid = GridSearchCV(GradientBoostingClassifier(), param_grid, cv=10, n_jobs=-1)

grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)


{}
GradientBoostingClassifier()
0.8219485412637008


In [27]:
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

#classifier = GradientBoostingClassifier(max_features=None, max_depth=3, criterion='friedman_mse')
classifier = GradientBoostingClassifier()
classifier.fit(x_train_chi, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

4704 470 784 1085
pod:  0.5805243445692884
pof:  0.09083880943177426
AUC:  0.744842767568757
accuracy:  0.82195087320744


In [28]:
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

classifier = GradientBoostingClassifier()
classifier.fit(X_train, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X_train, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

4717 457 779 1090
pod:  0.5831995719636169
pof:  0.0883262466177039
AUC:  0.7474366626729565
accuracy:  0.8245066023001562


In [29]:
df_x_train_chi = pd.DataFrame(x_train_chi)
df_x_train_chi.head()

#FNN
def get_FNN_Predict(X2_train, X2_test, y2_train, y2_test):
    from keras.models import Sequential
    from keras.layers import Dense
    #create model
    model = Sequential()
    #get number of columns in training data
    n_cols = X2_train.shape[1]
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    #compile model using mse as a measure of model performance
    #model.compile(optimizer='adam', loss='mean_squared_error')
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    #model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  
    from keras.callbacks import EarlyStopping
    #set early stopping monitor so the model stops training when it won't improve anymore
    early_stopping_monitor = EarlyStopping(patience=3)
    #train model
    model.fit(X2_train, y2_train, validation_split=0.2, epochs=30, callbacks=[early_stopping_monitor])
    y2_pred = model.predict(X2_test)
    print('y2_pred: ',y2_pred)
    y22_pred=y2_pred.round()
    print('y22_pred: ',y22_pred)
    return y22_pred

pod_list = []
pof_list = []
auc_val_list = []
tn_list= []
fp_list= []
fn_list= []
tp_list= []
    
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10)
for train_index, test_index in folds.split(df_x_train_chi,y_train):
    X2_train, X2_test, y2_train, y2_test=df_x_train_chi.iloc[train_index], df_x_train_chi.iloc[test_index],y_train.iloc[train_index], y_train.iloc[test_index]
    y_pred = get_FNN_Predict(X2_train, X2_test, y2_train, y2_test)
    tn, fp, fn, tp  = confusion_matrix(y2_test, y_pred).ravel()
    tn_list.append(tn)
    fp_list.append(fp)
    fn_list.append(fn)
    tp_list.append(tp)
    pod=tp/(tp+fn)
    print('pod 1st: ',pod)
    pof=fp/(fp+tn)
    auc_val=(1+pod-pof)/2
    #break
    pod_list.append(pod)
    pof_list.append(pof)
    auc_val_list.append(auc_val)

print('pod: ',pod_list)
print ('pof: ',pof_list)
print ('auc: ',auc_val_list)

print('tn_list: ',tn_list)
print ('fp_list: ',fp_list)
print ('fn_list: ',fn_list)
print ('tp_list: ',tp_list)


tn=sum(tn_list)
fp=sum(fp_list) 
fn=sum(fn_list) 
tp=sum(tp_list)
print(tn, fp, fn, tp)
 


Epoch 1/30
159/159 [==============================] - 6s 25ms/step - loss: 0.5260 - accuracy: 0.7704 - val_loss: 0.4505 - val_accuracy: 0.8021
Epoch 2/30
159/159 [==============================] - 1s 5ms/step - loss: 0.4115 - accuracy: 0.7968 - val_loss: 0.4355 - val_accuracy: 0.8210
Epoch 3/30
159/159 [==============================] - 1s 5ms/step - loss: 0.4141 - accuracy: 0.7966 - val_loss: 0.4172 - val_accuracy: 0.8099
Epoch 4/30
159/159 [==============================] - 1s 5ms/step - loss: 0.4001 - accuracy: 0.8110 - val_loss: 0.4130 - val_accuracy: 0.8139
Epoch 5/30
159/159 [==============================] - 1s 5ms/step - loss: 0.3831 - accuracy: 0.8136 - val_loss: 0.4048 - val_accuracy: 0.8210
Epoch 6/30
159/159 [==============================] - 1s 5ms/step - loss: 0.3770 - accuracy: 0.8273 - val_loss: 0.4086 - val_accuracy: 0.8233
Epoch 7/30
159/159 [==============================] - 1s 5ms/step - loss: 0.3884 - accuracy: 0.8191 - val_loss: 0.4167 - val_accuracy: 0.8226
Epoch

Epoch 1/30
159/159 [==============================] - 3s 9ms/step - loss: 0.5156 - accuracy: 0.7583 - val_loss: 0.4145 - val_accuracy: 0.8091
Epoch 2/30
159/159 [==============================] - 1s 5ms/step - loss: 0.4044 - accuracy: 0.8130 - val_loss: 0.4115 - val_accuracy: 0.8202
Epoch 3/30
159/159 [==============================] - 1s 5ms/step - loss: 0.4032 - accuracy: 0.8053 - val_loss: 0.4116 - val_accuracy: 0.8139
Epoch 4/30
159/159 [==============================] - 1s 5ms/step - loss: 0.4020 - accuracy: 0.8052 - val_loss: 0.4140 - val_accuracy: 0.8233
Epoch 5/30
159/159 [==============================] - 1s 5ms/step - loss: 0.3831 - accuracy: 0.8283 - val_loss: 0.4109 - val_accuracy: 0.8218
Epoch 6/30
159/159 [==============================] - 1s 5ms/step - loss: 0.3835 - accuracy: 0.8219 - val_loss: 0.4035 - val_accuracy: 0.8257
Epoch 7/30
159/159 [==============================] - 1s 5ms/step - loss: 0.3993 - accuracy: 0.8113 - val_loss: 0.4102 - val_accuracy: 0.8281
Epoch 

Epoch 1/30
159/159 [==============================] - 3s 8ms/step - loss: 0.5273 - accuracy: 0.7609 - val_loss: 0.4387 - val_accuracy: 0.8099
Epoch 2/30
159/159 [==============================] - 1s 5ms/step - loss: 0.4086 - accuracy: 0.8145 - val_loss: 0.4170 - val_accuracy: 0.8107
Epoch 3/30
159/159 [==============================] - 1s 5ms/step - loss: 0.4021 - accuracy: 0.8185 - val_loss: 0.4113 - val_accuracy: 0.8091
Epoch 4/30
159/159 [==============================] - 1s 5ms/step - loss: 0.4043 - accuracy: 0.8113 - val_loss: 0.4198 - val_accuracy: 0.8044
Epoch 5/30
159/159 [==============================] - 1s 5ms/step - loss: 0.3809 - accuracy: 0.8222 - val_loss: 0.4093 - val_accuracy: 0.8186
Epoch 6/30
159/159 [==============================] - 1s 5ms/step - loss: 0.3948 - accuracy: 0.8188 - val_loss: 0.4094 - val_accuracy: 0.8147
Epoch 7/30
159/159 [==============================] - 1s 5ms/step - loss: 0.3745 - accuracy: 0.8276 - val_loss: 0.4037 - val_accuracy: 0.8194
Epoch 

Epoch 1/30
159/159 [==============================] - 3s 8ms/step - loss: 0.5157 - accuracy: 0.7598 - val_loss: 0.4368 - val_accuracy: 0.8099
Epoch 2/30
159/159 [==============================] - 1s 5ms/step - loss: 0.4059 - accuracy: 0.8097 - val_loss: 0.4140 - val_accuracy: 0.8162
Epoch 3/30
159/159 [==============================] - 1s 5ms/step - loss: 0.3926 - accuracy: 0.8133 - val_loss: 0.4088 - val_accuracy: 0.8194
Epoch 4/30
159/159 [==============================] - 1s 5ms/step - loss: 0.3936 - accuracy: 0.8195 - val_loss: 0.4064 - val_accuracy: 0.8241
Epoch 5/30
159/159 [==============================] - 1s 5ms/step - loss: 0.3950 - accuracy: 0.8106 - val_loss: 0.4104 - val_accuracy: 0.8233
Epoch 6/30
159/159 [==============================] - 1s 5ms/step - loss: 0.3783 - accuracy: 0.8201 - val_loss: 0.4088 - val_accuracy: 0.8123
Epoch 7/30
159/159 [==============================] - 1s 5ms/step - loss: 0.3867 - accuracy: 0.8142 - val_loss: 0.4055 - val_accuracy: 0.8131
Epoch 

Epoch 1/30
159/159 [==============================] - 3s 10ms/step - loss: 0.5365 - accuracy: 0.7561 - val_loss: 0.4555 - val_accuracy: 0.8005
Epoch 2/30
159/159 [==============================] - 1s 5ms/step - loss: 0.4104 - accuracy: 0.8203 - val_loss: 0.4263 - val_accuracy: 0.8131
Epoch 3/30
159/159 [==============================] - 1s 5ms/step - loss: 0.4080 - accuracy: 0.8160 - val_loss: 0.4159 - val_accuracy: 0.8162
Epoch 4/30
159/159 [==============================] - 1s 5ms/step - loss: 0.3827 - accuracy: 0.8215 - val_loss: 0.4251 - val_accuracy: 0.8186
Epoch 5/30
159/159 [==============================] - 1s 5ms/step - loss: 0.3940 - accuracy: 0.8248 - val_loss: 0.4126 - val_accuracy: 0.8186
Epoch 6/30
159/159 [==============================] - 1s 5ms/step - loss: 0.3854 - accuracy: 0.8282 - val_loss: 0.4175 - val_accuracy: 0.8226
Epoch 7/30
159/159 [==============================] - 1s 5ms/step - loss: 0.3977 - accuracy: 0.8116 - val_loss: 0.4190 - val_accuracy: 0.8178
Epoch

Epoch 1/30
159/159 [==============================] - 3s 8ms/step - loss: 0.5095 - accuracy: 0.7522 - val_loss: 0.4129 - val_accuracy: 0.8202
Epoch 2/30
159/159 [==============================] - 1s 5ms/step - loss: 0.3976 - accuracy: 0.8106 - val_loss: 0.4063 - val_accuracy: 0.8241
Epoch 3/30
159/159 [==============================] - 1s 5ms/step - loss: 0.3707 - accuracy: 0.8287 - val_loss: 0.4002 - val_accuracy: 0.8265
Epoch 4/30
159/159 [==============================] - 1s 5ms/step - loss: 0.3786 - accuracy: 0.8205 - val_loss: 0.4027 - val_accuracy: 0.8226
Epoch 5/30
159/159 [==============================] - 1s 5ms/step - loss: 0.3785 - accuracy: 0.8241 - val_loss: 0.3964 - val_accuracy: 0.8273
Epoch 6/30
159/159 [==============================] - 1s 5ms/step - loss: 0.3863 - accuracy: 0.8209 - val_loss: 0.4118 - val_accuracy: 0.8068
Epoch 7/30
159/159 [==============================] - ETA: 0s - loss: 0.3695 - accuracy: 0.83 - 1s 5ms/step - loss: 0.3696 - accuracy: 0.8318 - val_

Epoch 1/30
159/159 [==============================] - 3s 8ms/step - loss: 0.5422 - accuracy: 0.7400 - val_loss: 0.4599 - val_accuracy: 0.7950
Epoch 2/30
159/159 [==============================] - 1s 5ms/step - loss: 0.4134 - accuracy: 0.8007 - val_loss: 0.4338 - val_accuracy: 0.8147
Epoch 3/30
159/159 [==============================] - 1s 5ms/step - loss: 0.4048 - accuracy: 0.8038 - val_loss: 0.4149 - val_accuracy: 0.8226
Epoch 4/30
159/159 [==============================] - 1s 5ms/step - loss: 0.3968 - accuracy: 0.8110 - val_loss: 0.4102 - val_accuracy: 0.8297
Epoch 5/30
159/159 [==============================] - 1s 5ms/step - loss: 0.3661 - accuracy: 0.8273 - val_loss: 0.4113 - val_accuracy: 0.8273
Epoch 6/30
159/159 [==============================] - 1s 5ms/step - loss: 0.3821 - accuracy: 0.8245 - val_loss: 0.4043 - val_accuracy: 0.8289
Epoch 7/30
159/159 [==============================] - 1s 5ms/step - loss: 0.3867 - accuracy: 0.8187 - val_loss: 0.4114 - val_accuracy: 0.8226
Epoch 

Epoch 1/30
159/159 [==============================] - 3s 8ms/step - loss: 0.5140 - accuracy: 0.7521 - val_loss: 0.4253 - val_accuracy: 0.8060
Epoch 2/30
159/159 [==============================] - 1s 5ms/step - loss: 0.4111 - accuracy: 0.8051 - val_loss: 0.4103 - val_accuracy: 0.8147
Epoch 3/30
159/159 [==============================] - 1s 5ms/step - loss: 0.3917 - accuracy: 0.8142 - val_loss: 0.4065 - val_accuracy: 0.8273
Epoch 4/30
159/159 [==============================] - 1s 5ms/step - loss: 0.3753 - accuracy: 0.8269 - val_loss: 0.4033 - val_accuracy: 0.8210
Epoch 5/30
159/159 [==============================] - 1s 5ms/step - loss: 0.3832 - accuracy: 0.8239 - val_loss: 0.4134 - val_accuracy: 0.8273
Epoch 6/30
159/159 [==============================] - 1s 5ms/step - loss: 0.3793 - accuracy: 0.8224 - val_loss: 0.4041 - val_accuracy: 0.8226
Epoch 7/30
159/159 [==============================] - 1s 5ms/step - loss: 0.3862 - accuracy: 0.8246 - val_loss: 0.4051 - val_accuracy: 0.8241
y2_pre

Epoch 1/30
159/159 [==============================] - 3s 8ms/step - loss: 0.5412 - accuracy: 0.7367 - val_loss: 0.4214 - val_accuracy: 0.8005
Epoch 2/30
159/159 [==============================] - 1s 5ms/step - loss: 0.4153 - accuracy: 0.8134 - val_loss: 0.4051 - val_accuracy: 0.8107
Epoch 3/30
159/159 [==============================] - 1s 5ms/step - loss: 0.4064 - accuracy: 0.8023 - val_loss: 0.4033 - val_accuracy: 0.8084
Epoch 4/30
159/159 [==============================] - 1s 5ms/step - loss: 0.3907 - accuracy: 0.8171 - val_loss: 0.3992 - val_accuracy: 0.8178
Epoch 5/30
159/159 [==============================] - 1s 5ms/step - loss: 0.3786 - accuracy: 0.8310 - val_loss: 0.3930 - val_accuracy: 0.8226
Epoch 6/30
159/159 [==============================] - 1s 5ms/step - loss: 0.3843 - accuracy: 0.8206 - val_loss: 0.3949 - val_accuracy: 0.8194
Epoch 7/30
159/159 [==============================] - 1s 5ms/step - loss: 0.3958 - accuracy: 0.8104 - val_loss: 0.3970 - val_accuracy: 0.8194
Epoch 

Epoch 1/30
159/159 [==============================] - 3s 8ms/step - loss: 0.5147 - accuracy: 0.7573 - val_loss: 0.4478 - val_accuracy: 0.7973
Epoch 2/30
159/159 [==============================] - 1s 5ms/step - loss: 0.4130 - accuracy: 0.7993 - val_loss: 0.4186 - val_accuracy: 0.8107
Epoch 3/30
159/159 [==============================] - 1s 5ms/step - loss: 0.3888 - accuracy: 0.8177 - val_loss: 0.4143 - val_accuracy: 0.8131
Epoch 4/30
159/159 [==============================] - 1s 5ms/step - loss: 0.4000 - accuracy: 0.8092 - val_loss: 0.4117 - val_accuracy: 0.8044
Epoch 5/30
159/159 [==============================] - 1s 5ms/step - loss: 0.3827 - accuracy: 0.8137 - val_loss: 0.4052 - val_accuracy: 0.8131
Epoch 6/30
159/159 [==============================] - 1s 5ms/step - loss: 0.3794 - accuracy: 0.8257 - val_loss: 0.4069 - val_accuracy: 0.8162
Epoch 7/30
159/159 [==============================] - 1s 5ms/step - loss: 0.3828 - accuracy: 0.8175 - val_loss: 0.4134 - val_accuracy: 0.8202
Epoch 

In [30]:
#RNN
def get_RNN_Predict(X2_train, X2_test, y2_train, y2_test):
    import pandas as pd
    from keras.models import Sequential
    from keras.layers import Dense,Dropout, LSTM, GRU
    from keras.layers import Embedding
    max_features = 10000 # number of words to consider as features
    import numpy as np
    #create model 
    model = Sequential()
    model.add(Embedding(max_features, 32))
    model.add(LSTM(32))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['acc'])
    history = model.fit(X2_train, y2_train, epochs=5, batch_size=128, validation_split=0.2)
    y2_pred = model.predict(X2_test)
    y22_pred=y2_pred.round()
    print('y22_pred: ',y22_pred)
    return y22_pred

pod_list = []
pof_list = []
auc_val_list = []
tn_list= []
fp_list= []
fn_list= []
tp_list= []
    
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10)
for train_index, test_index in folds.split(df_x_train_chi,y_train):
    X2_train, X2_test, y2_train, y2_test=df_x_train_chi.iloc[train_index], df_x_train_chi.iloc[test_index],y_train.iloc[train_index], y_train.iloc[test_index]
    y_pred = get_RNN_Predict(X2_train, X2_test, y2_train, y2_test)
    tn, fp, fn, tp  = confusion_matrix(y2_test, y_pred).ravel()
    tn_list.append(tn)
    fp_list.append(fp)
    fn_list.append(fn)
    tp_list.append(tp)
    
    pod=tp/(tp+fn)
    print('pod 1st: ',pod)
    pof=fp/(fp+tn)
    auc_val=(1+pod-pof)/2
    #break
    pod_list.append(pod)
    pof_list.append(pof)
    auc_val_list.append(auc_val)

print('pod: ',pod_list)
print ('pof: ',pof_list)
print ('auc: ',auc_val_list)

tn=sum(tn_list)
fp=sum(fp_list) 
fn=sum(fn_list) 
tp=sum(tp_list)
print(tn, fp, fn, tp)
 

Epoch 1/5
40/40 [==============================] - 9s 71ms/step - loss: 0.6199 - acc: 0.6979 - val_loss: 0.5658 - val_acc: 0.7279
Epoch 2/5
40/40 [==============================] - 1s 26ms/step - loss: 0.5385 - acc: 0.7451 - val_loss: 0.5105 - val_acc: 0.7547
Epoch 3/5
40/40 [==============================] - 1s 26ms/step - loss: 0.5214 - acc: 0.7557 - val_loss: 0.5052 - val_acc: 0.7658
Epoch 4/5
40/40 [==============================] - 1s 26ms/step - loss: 0.5211 - acc: 0.7470 - val_loss: 0.5138 - val_acc: 0.7618
Epoch 5/5
40/40 [==============================] - 1s 26ms/step - loss: 0.5074 - acc: 0.7707 - val_loss: 0.5008 - val_acc: 0.7658
y22_pred:  [[1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0

Epoch 1/5
40/40 [==============================] - 9s 72ms/step - loss: 0.6078 - acc: 0.7339 - val_loss: 0.5763 - val_acc: 0.7279
Epoch 2/5
40/40 [==============================] - 1s 27ms/step - loss: 0.5690 - acc: 0.7314 - val_loss: 0.5400 - val_acc: 0.7279
Epoch 3/5
40/40 [==============================] - 1s 27ms/step - loss: 0.5276 - acc: 0.7420 - val_loss: 0.5046 - val_acc: 0.7618
Epoch 4/5
40/40 [==============================] - 1s 27ms/step - loss: 0.5065 - acc: 0.7609 - val_loss: 0.5056 - val_acc: 0.7658
Epoch 5/5
40/40 [==============================] - 1s 27ms/step - loss: 0.4996 - acc: 0.7680 - val_loss: 0.4986 - val_acc: 0.7689
y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0

Epoch 1/5
40/40 [==============================] - 9s 63ms/step - loss: 0.6119 - acc: 0.7385 - val_loss: 0.5696 - val_acc: 0.7279
Epoch 2/5
40/40 [==============================] - 1s 27ms/step - loss: 0.5443 - acc: 0.7442 - val_loss: 0.5272 - val_acc: 0.7547
Epoch 3/5
40/40 [==============================] - 1s 27ms/step - loss: 0.5103 - acc: 0.7628 - val_loss: 0.5086 - val_acc: 0.7618
Epoch 4/5
40/40 [==============================] - 1s 27ms/step - loss: 0.4969 - acc: 0.7643 - val_loss: 0.5065 - val_acc: 0.7689
Epoch 5/5
40/40 [==============================] - 1s 27ms/step - loss: 0.4982 - acc: 0.7669 - val_loss: 0.5089 - val_acc: 0.7689
y22_pred:  [[0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0

Epoch 1/5
40/40 [==============================] - 9s 64ms/step - loss: 0.6187 - acc: 0.6821 - val_loss: 0.5664 - val_acc: 0.7279
Epoch 2/5
40/40 [==============================] - 1s 28ms/step - loss: 0.5484 - acc: 0.7412 - val_loss: 0.5201 - val_acc: 0.7516
Epoch 3/5
40/40 [==============================] - 1s 28ms/step - loss: 0.5113 - acc: 0.7603 - val_loss: 0.5254 - val_acc: 0.7547
Epoch 4/5
40/40 [==============================] - 1s 28ms/step - loss: 0.5066 - acc: 0.7621 - val_loss: 0.5039 - val_acc: 0.7618
Epoch 5/5
40/40 [==============================] - 1s 28ms/step - loss: 0.5080 - acc: 0.7630 - val_loss: 0.5144 - val_acc: 0.7571
y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0

Epoch 1/5
40/40 [==============================] - 9s 65ms/step - loss: 0.6151 - acc: 0.6881 - val_loss: 0.5730 - val_acc: 0.7248
Epoch 2/5
40/40 [==============================] - 1s 30ms/step - loss: 0.5609 - acc: 0.7316 - val_loss: 0.5578 - val_acc: 0.7271
Epoch 3/5
40/40 [==============================] - 1s 28ms/step - loss: 0.5300 - acc: 0.7521 - val_loss: 0.5088 - val_acc: 0.7666
Epoch 4/5
40/40 [==============================] - 1s 29ms/step - loss: 0.5025 - acc: 0.7566 - val_loss: 0.5056 - val_acc: 0.7658
Epoch 5/5
40/40 [==============================] - 1s 28ms/step - loss: 0.5042 - acc: 0.7674 - val_loss: 0.5041 - val_acc: 0.7658
y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0